Code to compare two 10-K reports from the two different datasets, results saved in folder "comparison"

In [1]:
import pandas as pd
import os
from glob import glob
import random
import pyarrow.parquet as pq

In [2]:
# Load the first dataset
final_data_path = "data/preprocessed.parquet"
parquet_file = pq.ParquetFile(final_data_path)

all_data = pd.concat([batch.to_pandas() for batch in parquet_file.iter_batches(batch_size=8)], ignore_index=True)
print(all_data.head())

  ticker      cik formType                    filedAt  \
0   PDRO  1627554     10-K  2016-01-29T16:34:46-05:00   
1   None   815093     10-K  2016-01-29T16:59:45-05:00   
2   None  1598893     10-K  2016-01-29T18:26:19-05:00   
3   WCUI  1516887     10-K  2016-01-29T17:30:16-05:00   
4    JEF  1084580     10-K  2016-01-29T16:59:43-05:00   

                                           filingUrl  \
0  https://www.sec.gov/Archives/edgar/data/162755...   
1  https://www.sec.gov/Archives/edgar/data/815093...   
2  https://www.sec.gov/Archives/edgar/data/159889...   
3  https://www.sec.gov/Archives/edgar/data/151688...   
4  https://www.sec.gov/Archives/edgar/data/108458...   

                                           section_1  \
0   ITEM 1. BUSINESS \n\n\n\nGeneral \n\n\n\nWe a...   
1   Item 1. Business. \n\n \n\nUnless the context...   
2   ITEM 1. BUSINESS\n\nBackground and Summary\n\...   
3   ITEM 1. \n\nBUSINESS \n\nBackground. \n\nWell...   
4   Item 1. \n\nBusiness. \n\n##TABLE_EN

In [26]:
random_row = all_data.sample(n=1)

# Write the row to a CSV file
output_folder = "comparison"
os.makedirs(output_folder, exist_ok=True)
output_file = os.path.join(output_folder, "steven_data.txt")
random_row.to_csv(output_file, index=False)


# Extract CIK and filedAt from the chosen row
cik = random_row['cik'].values[0]
final_date = random_row['filedAt'].values[0]

# Print the extracted values
print(f"CIK: {cik}, Filed At: {final_date}")

# Construct the path to the equivalent 10-K file based on cik and final_date
year = final_date[:4]
quarter = f"QTR{((int(final_date[5:7]) - 1) // 3) + 1}"
formatted_date = final_date[:10].replace("-", "")
file_name_start = f"{formatted_date}_10-K_edgar_data_{cik}"
base_path = "/home/caro/DS_complexity/data"
file_folder = os.path.join(base_path, year, quarter)

matching_files = [f for f in os.listdir(file_folder) if f.startswith(file_name_start) and f.endswith(".txt")]
if matching_files:
    file_path = os.path.join(file_folder, matching_files[0])
else:
    file_path = None

print(file_path)

if file_path:
    with open(file_path, 'r') as file:
        file_contents = file.read()

    # Save the contents to miguel_data.txt
    with open("comparison/original_data.txt", 'w') as output_file:
        output_file.write(file_contents)
else:
    print("File path is None. Cannot read the file.")

CIK: 1333822, Filed At: 2018-11-26T06:07:18-05:00
/home/caro/DS_complexity/data/2018/QTR4/20181126_10-K_edgar_data_1333822_0001564590-18-030063.txt


In [27]:
import csv
from difflib import unified_diff
from difflib import SequenceMatcher

# Read the content of the two CSV files
with open("comparison/steven_data.txt", 'r') as steven_file:
    steven_data = steven_file.read()

with open("comparison/original_data.txt", 'r') as original_file:
    original_data = original_file.read()

# Find the text in original_data that is not in steven_data
# Generate a unified diff between the two texts
# Tokenize the text into words
steven_tokens = steven_data.split()
original_tokens = original_data.split()

# Use SequenceMatcher to find differences
matcher = SequenceMatcher(None, steven_tokens, original_tokens)
diff_text = []
for tag, i1, i2, j1, j2 in matcher.get_opcodes():
    if tag != 'equal':
        diff_text.append(f"{tag}: Steven[{i1}:{i2}] -> Original[{j1}:{j2}]")
        diff_text.append(f"Steven: {' '.join(steven_tokens[i1:i2])}")
        diff_text.append(f"Original: {' '.join(original_tokens[j1:j2])}")

# Save the diff text to a CSV file
diff_file_path = "comparison/diff.txt"
with open(diff_file_path, 'w') as diff_file:
    for line in diff_text:
        diff_file.write(line + '\n')

print(f"Differences saved to {diff_file_path}")
# Separate the diff text based on labels and save them into different files
insert_text = []
delete_text = []
replace_text = []

for tag, i1, i2, j1, j2 in matcher.get_opcodes():
    if tag == 'insert':
        insert_text.append(f"Original[{j1}:{j2}]: {' '.join(original_tokens[j1:j2])}")
    elif tag == 'delete':
        delete_text.append(f"Steven[{i1}:{i2}]: {' '.join(steven_tokens[i1:i2])}")
    elif tag == 'replace':
        replace_text.append(f"Steven[{i1}:{i2}]: {' '.join(steven_tokens[i1:i2])} -> Original[{j1}:{j2}]: {' '.join(original_tokens[j1:j2])}")

# Save insert text
insert_file_path = "comparison/insert.txt"
with open(insert_file_path, 'w') as insert_file:
    for line in insert_text:
        insert_file.write(line + '\n')

# Save delete text
delete_file_path = "comparison/in_steven.txt"
with open(delete_file_path, 'w') as delete_file:
    for line in delete_text:
        delete_file.write(line + '\n')

# Save replace text
replace_file_path = "comparison/in_original.txt"
with open(replace_file_path, 'w') as replace_file:
    for line in replace_text:
        replace_file.write(line + '\n')

print(f"Insert differences saved to {insert_file_path}")
print(f"Delete differences saved to {delete_file_path}")
print(f"Replace differences saved to {replace_file_path}")


Differences saved to comparison/diff.txt
Insert differences saved to comparison/insert.txt
Delete differences saved to comparison/in_steven.txt
Replace differences saved to comparison/in_original.txt
